# <center>Data Science Training</center>
<center><b>Cross Validation Template</b><br>
Jiannan, 2016</center>

In [ ]:
import numpy as np
import pandas as pd

import sklearn as sk
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,average_precision_score,roc_curve,auc,precision_recall_curve
from sklearn.metrics import log_loss
%matplotlib inline
import matplotlib.pyplot as plt

import xgboost as xgb
from scipy import interp

import random

from __future__ import division
from __future__ import print_function

# Define fucntions to calculate feature importances in different criteria

In [ ]:
def get_importance(_bst, _importance_type):
    # if it's weight, then omap stores the number of missing values
    fmap = ''
    if _importance_type == 'weight':
        # do a simpler tree dump to save time
        trees = _bst.get_dump(fmap, with_stats=False)

        fmap = {}
        for tree in trees:
            for line in tree.split('\n'):
                # look for the opening square bracket
                arr = line.split('[')
                # if no opening bracket (leaf node), ignore this line
                if len(arr) == 1:
                    continue

                # extract feature name from string between []
                fid = arr[1].split(']')[0].split('<')[0]

                if fid not in fmap:
                    # if the feature hasn't been seen yet
                    fmap[fid] = 1
                else:
                    fmap[fid] += 1

        return fmap

    else:
        trees = _bst.get_dump(fmap, with_stats=True)

        _importance_type += '='
        fmap = {}
        gmap = {}
        for tree in trees:
            for line in tree.split('\n'):
                # look for the opening square bracket
                arr = line.split('[')
                # if no opening bracket (leaf node), ignore this line
                if len(arr) == 1:
                    continue

                # look for the closing bracket, extract only info within that bracket
                fid = arr[1].split(']')

                # extract gain or cover from string after closing bracket
                g = float(fid[1].split(_importance_type)[1].split(',')[0])

                # extract feature name from string before closing bracket
                fid = fid[0].split('<')[0]

                if fid not in fmap:
                    # if the feature hasn't been seen yet
                    fmap[fid] = 1
                    gmap[fid] = g
                else:
                    fmap[fid] += 1
                    gmap[fid] += g

        # calculate average value (gain/cover) for each feature
        for fid in gmap:
            gmap[fid] = gmap[fid] / fmap[fid]

        return gmap

# Read files and preparation before cross-validation

In [ ]:
dataset_modelA_clean = pd.read_csv("input_file_path",sep="|",na_values=["\N", "NULL"])

In [ ]:
ALL_TARGETS = ['target_all']

In [ ]:
targeting_features_file = 'feature_columns_metadata_path'
feature_df_targeting = pd.read_csv(targeting_features_file,na_values=["\N","NULL"])

features_columns_targeting = feature_df_targeting.targeting_features.tolist()

print(len(features_columns_targeting))

In [ ]:
# get data set for targeting
dataset_modelA_clean_targeting = dataset_modelA_clean[np.concatenate([features_columns_targeting, ALL_TARGETS])].copy()

In [ ]:
print(ALL_TARGETS)

for target in ALL_TARGETS:
    print("target number %s"%target)
    print(dataset_modelA_clean_targeting[target].sum())
    print("target proportion %s"%target)
    print(dataset_modelA_clean_targeting[target].mean())

# Start CV for Xgboost

In [ ]:
def run_cross_validation(_df, _classifier, _features_columns):
    # cross validation type can be changed here
    ss = sk.cross_validation.ShuffleSplit(len(_df.svocmasterid.unique()), n_iter=3, test_size=.3, random_state=0)
    target='target_all'
    prob_of = 'prob_of_all'
    
    results_cv_targeting = pd.DataFrame([], columns=['svocmasterid', target, 'fold', prob_of])

    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    all_tpr = []
    mean_lift = []
    mean_tp = 0.0
    mean_fp = range(0, 101)

    nb_calls_cv = pd.DataFrame([],columns=['nb_contacts', 'total_population', 'total_pos_targets', 'nb_pos_targets', 'pos_rate', 
                                           'Percentage_of_pos_targets_found', 'Percentage_of_Population', 'Lift'])
    feature_importances = pd.DataFrame([], columns=['feature', 'importance', 'fold'])

    fig = plt.figure(figsize=(6, 12))
    fig.subplots_adjust(bottom=-0.5, left=-0.5, top=0.5, right=1.5)


    print ('modeling started')

    for i, (train_index, valid_index) in enumerate(ss):

        customer_id = _df.svocmasterid.unique().copy()
        shuffled_customer_id = np.array(sorted(customer_id, key=lambda k: random.random()))
        train_customer_id = shuffled_customer_id[train_index]
        valid_customer_id = shuffled_customer_id[valid_index]

        train = _df.loc[ _df.svocmasterid.isin(train_customer_id), np.concatenate([_features_columns, [target]],
                        axis=0)].copy().reset_index(drop=True)
        valid = _df.loc[_df.svocmasterid.isin(valid_customer_id), np.concatenate([_features_columns, [target]],
                        axis=0)].copy().reset_index(drop=True)

        temp = valid[['svocmasterid', target]].copy()
        temp['fold'] = i

        # modeling#
        train_X = train.drop(['svocmasterid', target], axis=1)
        valid_X = valid.drop(['svocmasterid', target], axis=1)

        train_Y = np.array(train[target].astype(np.uint8))
        valid_Y = np.array(valid[target].astype(np.uint8))

        probas_ = _classifier.fit(train_X, train_Y,eval_metric='auc', eval_set=[(valid_X, valid_Y)],early_stopping_rounds=40).predict_proba(valid_X) 
        probabilities = pd.DataFrame(data=probas_[:, 1], index=valid_X.index, columns=[prob_of])

        temp = temp.join(probabilities, how='left')
        results_cv_targeting = results_cv_targeting.append(temp)

        ###############################################################################
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = sk.metrics.roc_curve(valid_Y, probas_[:, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = sk.metrics.auc(fpr, tpr)


        plt.subplot(2, 2, 1)
        plt.plot(fpr, tpr, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

        ###############################################################################
        # compute lift at 10%#
        sorted_proba = np.array(list(reversed(np.argsort(probas_[:, 1]))))
        X_test = valid_X
        y_test = valid_Y
        centile = X_test.shape[0] / 100
        positives = sum(y_test)
        lift = [0]
        for q in xrange(1, 101):
            if q == 100:
                tp = sum(np.array(y_test)[sorted_proba[(q - 1) * X_test.shape[0] / 100:X_test.shape[0]]])
            else:
                tp = sum(
                    np.array(y_test)[sorted_proba[(q - 1) * X_test.shape[0] / 100:q * X_test.shape[0] / 100]])
            lift.append(lift[q - 1] + 100 * tp / float(positives))
        quantiles = range(0, 101)
        mean_tp += interp(mean_fp, mean_fp, lift)
        mean_tp[0] = 0.0
        mean_lift.append(lift[10] / 10.)


        plt.subplot(2, 2, 2)
        plt.plot(quantiles, lift, label='Lift fold %d at 10 = %0.2f' % (i, lift[10] / 10.))
        print ('shuffle: %i, AUC: %f, lift at 10 percent: %f' % (i, roc_auc, lift[10] / 10.))
        
        ###############################################################################
        # Calculate nb contacts to make
        nb_calls = temp[['target_all','prob_of_all','fold']].copy()
        nb_calls = nb_calls.sort_values(by='prob_of_all', ascending=False).reset_index(drop=True)
        nb_calls['cum_Xsellers'] = np.cumsum(nb_calls.target_all)
        nb_calls = nb_calls.reset_index(drop=False)
        nb_calls = nb_calls.rename(columns={'index':'rank'})
        nb_calls['nb_contacts_100'] = nb_calls.loc[nb_calls.cum_Xsellers==100,'rank'].min()
        nb_calls['nb_contacts_200'] = nb_calls.loc[nb_calls.cum_Xsellers==200,'rank'].min()
        nb_calls['nb_contacts_500'] = nb_calls.loc[nb_calls.cum_Xsellers==500,'rank'].min()
        nb_calls['nb_contacts_1000'] = nb_calls.loc[nb_calls.cum_Xsellers==1000,'rank'].min()
        nb_calls['nb_contacts_2000'] = nb_calls.loc[nb_calls.cum_Xsellers==2000,'rank'].min()
        nb_calls['nb_contacts_3000'] = nb_calls.loc[nb_calls.cum_Xsellers==3000,'rank'].min()
        nb_calls['nb_contacts_all'] = nb_calls.loc[nb_calls.cum_Xsellers==nb_calls.cum_Xsellers.max(),'rank'].min()
        nb_calls = nb_calls[['nb_contacts_100','nb_contacts_200', 'nb_contacts_500','nb_contacts_1000', 'nb_contacts_2000','nb_contacts_3000','nb_contacts_all']].min()
        nb_calls = pd.DataFrame(nb_calls,columns=['nb_contacts'])
        nb_calls['total_population'] = temp.shape[0]
        nb_calls['total_pos_targets'] = temp.target_all.sum()
        nb_calls['nb_pos_targets']=[100,200,500,1000,2000,3000, temp.target_all.sum()]
        nb_calls['pos_rate'] = nb_calls.nb_pos_targets/nb_calls.nb_contacts
        nb_calls['Percentage_of_pos_targets_found'] = nb_calls.nb_pos_targets/nb_calls.total_pos_targets
        nb_calls['Percentage_of_Population'] = nb_calls.nb_contacts/nb_calls.total_population
        nb_calls['Lift'] = nb_calls.Percentage_of_pos_targets_found/nb_calls.Percentage_of_Population

        nb_calls_cv = nb_calls_cv.append(nb_calls)
        
        ###############################################################################
        feature_importances_data = []
        features = train_X.columns
        for feature_name, feature_importance in get_importance(_classifier.booster(), 'gain').iteritems():
            feature_importances_data.append({
                'feature': feature_name,
                'importance': feature_importance
            })

        temp = pd.DataFrame(feature_importances_data)
        temp['fold'] = i
        feature_importances = feature_importances.append(temp)
        
    
    nb_calls_cv = nb_calls_cv.reset_index().groupby('index').mean().sort_values(by='nb_pos_targets')
    results_cv_targeting = results_cv_targeting.reset_index(drop=True)
    
    feature_importances = feature_importances.groupby('feature')['importance'].agg([np.mean, np.std])
    feature_importances = feature_importances.sort_values(by='mean')
    feature_importances = feature_importances.reset_index()

    plt.subplot(2, 2, 1)
    mean_tpr /= len(ss)
    mean_tpr[-1] = 1.0
    mean_auc = sk.metrics.auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--', label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC', fontsize=14)
    plt.grid(True)
    plt.legend(loc="lower right")

    plt.subplot(2, 2, 2)
    mean_tp /= len(ss)
    mean_tp[-1] = 100.0
    mean_lift10 = np.mean(mean_lift)
    print('Mean AUC: %f, Mean lift at 10 percent: %f' % (mean_auc, mean_lift10))
    plt.plot(mean_fp, mean_tp, 'k--', label='Mean lift at 10 = %0.2f' % mean_lift10, lw=2)

    plt.plot([0, 100], [0, 100], 'k--', color=(0.6, 0.6, 0.6))
    plt.xlim([-5, 105])
    plt.ylim([-5, 105])
    plt.xlabel('Percentage of population')
    plt.ylabel('Cumulative gain')
    plt.title('Lift', fontsize=14)
    plt.grid(True)
    plt.legend(loc="lower right")

    plt.show()

    return results_cv_targeting, feature_importances, nb_calls_cv

In [ ]:
# parameters of the classifier need to be changed according to datasets
classifier = xgb.XGBClassifier(objective='binary:logistic',max_depth=6,n_estimators=400, learning_rate=0.05,max_delta_step=1,
                        min_child_weight=25, gamma=0.1, scale_pos_weight=1, colsample_bytree=0.85, subsample=0.85,colsample_bylevel=0.85,
                        nthread=10, seed=27)

results_cv_targeting, feature_importances, nb_calls_cv = run_cross_validation(dataset_modelA_clean_targeting, classifier , features_columns_targeting) 

In [ ]:
feature_importances_sort = feature_importances.sort_values(by='mean',ascending=False)

feature_importances_sort['relative_imp'] = 100.0 * (feature_importances_sort['mean'] / feature_importance_sort['mean'].max())
#feature_importances_sort['mean']/feature_importances_sort['mean'].sum()
feature_importances_sort.to_csv('file_path_to_store_feat_imp',sep='|', index=False)

In [ ]:
feature_importances_top10 = feature_importances_sort[:10][::-1].reset_index(drop=True)

In [ ]:
plt.figure(figsize=(10, 20))
plt.title("Top10 Most Important Feature importances for Model")
plt.barh(feature_importances_top10.index, feature_importances_top10['relative_imp'],
         color='#348ABD', align="center", lw='3', edgecolor='#348ABD', alpha=0.6)
plt.yticks(feature_importances_top10.index, feature_importances_top10['feature'], fontsize=12,)
plt.ylim([-1, feature_importances_top10.index.max()+1])
plt.xlim([0, feature_importances_top10['relative_imp'].max()*1.1])
plt.show()